In [70]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
import os

import citrine
from citrine import Citrine

from citrine.seeding.find_or_create import find_or_create_project, find_or_create_dataset
from citrine.jobs.waiting import wait_while_validating, wait_while_executing

# Testing

## AutoConfigure Workflow

In [90]:
import warnings
from itertools import islice

from citrine.exceptions import Unauthorized, NotFound, CitrineException

MAX_DATASETS = 50


def _own_dataset(project, dataset) -> bool:
    """Only the 'owning' project can change acls, so try that to check ownership"""
    try:
        project.make_private(dataset)
        return True
    except Unauthorized:
        return False


def cleanup_contents(project, strict=False):
    """
    Cleanup the contents of a project, inc modules, workflows, datasets but not GEM tables

    :param project: to clean up
    :param strict: whether to raise exceptions if a resource that should be deletable cannot be deleted
    :return: whether the project was cleaned to the point that it can be safely deleted
    """
    project_clean = True

    # Note that these are implemented but currently soft deletes
    for workflow in project.design_workflows.list():
        project.design_workflows.archive(workflow.uid)

    # Note that these are implemented but currently soft deletes
    for module in project.modules.list():
        project.modules.delete(module.uid)

    # We still delete projects even if they contain undeleteable definitions and tables.
    # From @djack:
    # "GEM tables cannot be deleted. I think orphaning them should be relatively safe to do."
    #
    # Don't delete projects that contain ara definitions or tables that are not deleteable yet
    # for defn in project.table_configs.list():
    #     # TODO delete table configs
    #     pass

    # Because data objects can depend on data objects from other datasets, we collect all of the
    # data objects to delete, sort them by the opposite of their write order, try deleting them
    # and then go back and delete the datasets that contained them

    errs_to_raise = []
    datasets_to_delete = []  # list of datasets to delete

    for dataset in islice(project.datasets.list(), MAX_DATASETS):
        dataset_clean = True

        # Skip datasets that are owned by other projects
        if not _own_dataset(project, dataset):
            continue

        # We don't block file deletion on references, so we can do this first
        for file_link in dataset.files.list():
            dataset.files.delete(file_link)

        deletion_failures = dataset.delete_contents(polling_delay=0.25)
        for (obj_id, failure) in deletion_failures:
            if strict:
                errs_to_raise.append(Exception(failure.message))

            warnings.warn(f"Failure in deleting GEMD Object while cleaning dataset: {dataset.uid}:"
                          f" {failure.message}")

        # If we expect to clean the dataset, add it to the delete list
        if len(deletion_failures) == 0:
            datasets_to_delete.append(dataset)
        else:
            # If we cannot delete all the datasets, then don't delete the project
            project_clean = False

    for dataset in datasets_to_delete:
        try:
            project.datasets.delete(dataset.uid)
        except CitrineException as err:
            project_clean = False
            if strict:
                errs_to_raise.append(err)
            else:
                warnings.warn("Failed to delete {} with uid=".format(dataset.name, dataset.uid), RuntimeWarning)

    # Raise any collected errors
    if len(errs_to_raise) > 0:
        raise Exception(errs_to_raise)

    return project_clean

In [98]:
from citrine.informatics.predictor_evaluator import CrossValidationEvaluator
from citrine.informatics.predictor_evaluation_metrics import RMSE, PVA
from citrine.informatics.workflows import PredictorEvaluationWorkflow

from citrine.builders.auto_configure import AutoConfigureWorkflow

In [99]:
client = Citrine(host=os.environ['DEV_CITRINE_API_HOST'], api_key=os.environ['DEV_CITRINE_API_KEY'])

project = client.projects.get('2e2b8001-6780-41af-88bd-df8a4d9fe59b')

In [67]:
auto_config = AutoConfigureWorkflow(project=project, name='Rosenbrock v14')

In [68]:
predictor = project.predictors.get('883045d6-e092-423f-a1b1-70e9cdd6b39f')

In [69]:
auto_config._predictor_evaluation_stage(predictor=predictor, evaluator=None, print_status_info=True)

Configuring predictor evaluation workflow/execution...
Status = SUCCEEDED                Elapsed time  =  74s
Status info:
['PredictorEvaluationWorkflow contains 1 evaluators.',
 'Evaluator names are distinct.',
 'At least 2 cross-validation folds requested.',
 'At least one cross-validation trial requested.']


In [65]:
auto_config.predictor_evaluation_execution